## Generative Advertial Network
- 이안 굿펠로우 제안
- 내부의 두 신경망이 상호 경쟁하면서 학습
    - 생성망, 판별망
- real data(실제 데이터)와 비슷한 확률분포를 가지는 fake data 생성
- fake data(허구 데이터)는 GAN에서 만들어진 '생성 데이터'
- GAN 예시: 실제 데이터로 얼굴 사진을 제공 -> 비슷한 확률분포를 가지는 새로운 fake 얼굴 사진 생성
    - 비교) 데이터의 레이블을 학습하는 판별형 신경망 DNN, CNN, ...
- **입력데이터**: 무작위 잡음
- **출력데이터**: 입력 데이터보다 높은 차원
    - 특정 분포도를 갖는 데이터
    - ex) 필기체 숫자, 사람 얼굴 사진
- **학습이후**: GAN에 새로운 무작위 노이즈 입력 -> 학습한 실제 데이터와 유사한 형태의 fake data 출력

### 보충자료
- https://zzsza.github.io/data/2017/12/27/gan/

## GAN의 구조
- 두 네트워크가 복합적으로 구성되어있음
    - 하나는 **생성망**
    - 하나는 **판별망**
- **학습 목적**: 학습한 실제 데이터와 같은 확률분포를 가지는 새로운 fake data를 만들도록 **'생성망'**을 학습
    - 생성망: 주어진 데이터와 유사항 fake data 생성
        - 저차원의 무작위 노이즈 벡터(Z)를 입력받아 고차원의 fake data 생성
        - 실제 이미지를 학습하여 실제 이미지와 확률분포가 비슷한 fake data 생성
            - 판별망의 판단 결과 활용
        - fake data가 신경망의 학습이 진행됨에 따라 점점 real data와 유사해짐
    - 판별망: 생성망에서 전달된 데이터가 fake 인지 real 인지 구분
        - 생성망의 결과를 판별하는 신경망
        - 이미지 자체를 판단하는게 아니라, 이미지의 **확률분포를 판별**함
        - 확률분포 차이를 판별하도록 학습

## 판별망 동작 원리
- real data를 1로 판별
    1. 실제 데이터 일부(batch)로 부터 가져온 샘플, 판별망에 입력
    2. 미분 가능한 **판별 함수 D** (신경망으로 구성됨)
    3. 1을 출력하려고 노력하는 **판별함수 D**
- fake date를 0으로 판별
    1. 무작위 **노이즈 벡터 Z** (임의의 확률분포를 가짐)
    2. Z를 미분 가능한 **생성 함수 G**에 통과시키기 (신경망으로 구성)
    3. 모델로부터 가져온 샘플
    4. 미분 가능한 **판별 함수 D**
    5. 0을 출력하려고 노력하는 **판별 함수 D**

## 학습 과정
1. 판별망 학습
    - 판별 값은 1(real), 0(fake)
    - 판별망을 학습시킬 때 생성망이 학습되면 안 됨(가중치 고정)
    - 판별망에 들어가는 입력은, 실제 데이터에서 추출한 배치 데이터 + 생성망에서 만든 fake 데이터
    - 목표 출력 값은 각각 군 별로 1 or 0
2. 생성망 학습
    - 생성망의 결과가 판별망으로 들어가도록 **가상 신경망** 모델 구성
        - '생성망 최적화를 위한 학습용 생성 함수 GD' or '학습용 생성망'이라고 부름
        - 판별망 부분 학습되지 않도록 가중치 고정
        - 이후, Z로 부터 생성된 fake data가 얼마나 real data에 유사한지 판별한 결과를 내게 되고,
        - 그 결과가 모두 1이 되도록 생성망을 학습
            - 0 or 1 구분 결과는 크로스 엔트로피로 표현됨(분류망에서 처럼)
 
 
### 이렇게 한번씩 실행 되면 GAN 학습 한 번 수행된 셈